In [28]:
# табличное представление данных
import pandas as pd
import numpy as np

# стемминг
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize

import string
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import os

In [29]:
family = pd.read_excel('true_vent_family_check.xlsx', header=0)
grand= pd.read_excel('all_vent_new_send.xlsx', header=0)

In [30]:
family.head()


,Воздухо - производительность,Напор,Название,GrandTenderHash
0,150,150,"Тип исполнения крышный, уличного исполнения Вы...",4153669e-179a-2ff5-8678-ac8d3199e248
1,150,150,Крышная вытяжная установка уличного исполнения...,4153669e-179a-2ff5-8678-ac8d3199e248
2,150,150,Вытяжной вентиляционный агрегат крышного испол...,4153669e-179a-2ff5-8678-ac8d3199e249
3,150,150,Вытяжной вентиляционный агрегат крышного испол...,4153669e-179a-2ff5-8678-ac8d3199e250
4,150,150,Вентиляционна вытяжная установка крышного испо...,4153669e-179a-2ff5-8678-ac8d3199e251


In [31]:
grand.head()

,id,Хэш-код,Название,Воздухо - производительность,Напор
0,26558,b4ceac0d-23e8-952c-e8e8-83b43d5db298,Тип исполнения крышное Вытяжной вентилятор в к...,2630,120
1,26557,39b625d7-cd5f-68ba-64b0-a3298fbb0603,Тип исполнения подвесной Канальные установки в...,168,110
2,26556,cbee6473-bead-d32d-97a5-437b2db61bb3,Тип исполнения подвесной Канальные установки в...,440,120
3,26555,d9b50bd7-4f1c-d80e-a457-801adf2c65d5,Тип исполнения подвесной Канальные установки в...,510,160
4,26554,269a68a7-1664-90a4-d96c-939518fc77d1,Тип исполнения подвесной Канальные установки в...,620,160


In [32]:
grand = grand.astype(str)
grand['Название_all'] = grand[[
    'Воздухо - производительность',
    'Напор',
    'Название'
                              ]].agg(' '.join, axis=1)
grand['Название_all'][100]

'960 300 Тип исполнения крышный, уличного исполнения Вытяжные установки в комплекте ***: 1)клапан воздушный, 2)секция шумоглушителя на строне всасывания, 3)секция вентилятора, с частотным преобразователем, 4)секция шумоглушителя на сторне выброса, 5) шкаф управления ** 6) Частотный преобразователь (можно в шкафу управления)'

In [33]:
def tokenize_num(file_text, allowed_postags=['NOUN', 'ADJF', 'ADJS', 'VERB', 'ADVB'], non_allowed_postags = ['PREP', "CONJ"]):
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)
    
    # let's delete punctuation symbols
    punctuation = string.punctuation.replace("-", "")
    tokens = [i for i in tokens if (i not in punctuation)]
    tokens = [i for i in tokens if (i not in 'nan')]
    
    tokens = [re.sub(r"[#%!@*.,`']", " ", sentence) for sentence in tokens]
#     tokens = [' '.join(map(str.strip,i.split())) for i in tokens]
    
    # поменяй комментарии. оставить только числа, оставить только текст
    tokens = [' '.join(re.findall(r'[A-Za-z]+|\d+', token)) for token in tokens]
#     tokens = [re.sub(r'[^\w\s]+|[\d]+', r'',i).strip() for i in tokens]
    
    stemmer = SnowballStemmer("russian") 
    tokens = [i for i in tokens if morph.parse(i)[0].tag.POS not in non_allowed_postags]
    tokens = [stemmer.stem(i) for i in tokens if (i not in punctuation)]
    return tokens 

In [34]:
def tokenize_text(file_text, allowed_postags=['NOUN', 'ADJF', 'ADJS', 'VERB', 'ADVB'], non_allowed_postags = ['PREP', "CONJ"]):
    file_text = re.sub(r"[-_]", " ", file_text)
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)
    
    # let's delete punctuation symbols
    punctuation = string.punctuation # .replace("-", "")
    tokens = [i for i in tokens if (i not in punctuation)]
    tokens = [i for i in tokens if (i not in 'nan')]
    
    # заменяет символы в квадратных скобках на пробелы
    tokens = [re.sub(r"[#%!@*.,`'-]", " ", sentence) for sentence in tokens]
#     tokens = [' '.join(map(str.strip,i.split())) for i in tokens]
    
    # поменяй комментарии. оставить только числа, оставить только текст
#     tokens = [' '.join(re.findall(r'[A-Za-z]+|\d+', token)) for token in tokens]
    tokens = [re.sub(r'[^\w\s]+|[\d]+', r'',i).strip() for i in tokens]
    
    stemmer = SnowballStemmer("russian") 
    tokens = [i for i in tokens if morph.parse(i)[0].tag.POS not in non_allowed_postags]
    tokens = [stemmer.stem(i) for i in tokens if (i not in punctuation)]
    return tokens 

In [35]:
grand['Название_all'] = grand['Название_all'].astype(str)

In [36]:
grand['Название_num'] = grand['Название_all']
grand['Название_num'] = grand['Название_num'].apply(tokenize_num)
grand['Название_num']

0                                [2630, 120, 1, 2, 3, 4]
1                                 [168, 110, 1, 2, 3, 4]
2                              [440, 120, 1, 2, 3, 4, 5]
3                              [510, 160, 1, 2, 3, 4, 5]
4                              [620, 160, 1, 2, 3, 4, 5]
                             ...                        
239    [8900, 500, 1, 2, 3, G 4, 4, 5, 95 70, 90 70, ...
240    [11300, 550, 1, 2, 3, G 4, 4, 5, 95 70, 90 70,...
241    [13000, 500, 1, 2, 3, G 4, 4, 5, 95 70, 90 70,...
242    [16200, 850, 1, 2, 3, G 4, 4, 5, 95 70, 90 70,...
243    [18500, 750, 1, 2, 3, G 4, 4, 5, 95 70, 90 70,...
Name: Название_num, Length: 244, dtype: object

In [37]:
grand['Название_text'] = grand['Название_all']
grand['Название_text'] = grand['Название_text'].apply(tokenize_text)
grand['Название_text']

0      [тип, исполнен, крышн, вытяжн, вентилятор, ком...
1      [тип, исполнен, подвесн, канальн, установк, ко...
2      [тип, исполнен, подвесн, канальн, установк, ко...
3      [тип, исполнен, подвесн, канальн, установк, ко...
4      [тип, исполнен, подвесн, канальн, установк, ко...
                             ...                        
239    [тип, исполнен, напольн, приточн, вытяжн, уста...
240    [тип, исполнен, напольн, приточн, вытяжн, уста...
241    [тип, исполнен, напольн, приточн, вытяжн, уста...
242    [тип, исполнен, напольн, приточн, вытяжн, уста...
243    [тип, исполнен, напольн, приточн, вытяжн, уста...
Name: Название_text, Length: 244, dtype: object

In [38]:
family['Название'] = family['Название'].astype(str)

In [39]:
family['Название_num'] = family['Название']
family['Название_num'] = family['Название_num'].apply(tokenize_num)
family['Название_num']

0          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
1     [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50, NED]
2          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
3          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
4          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
5          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
6          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
7          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
8          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
9          [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
10         [1, 2, 3, 4, 5, 6, 150 00, 3, 150, 0 50]
Name: Название_num, dtype: object

In [40]:
family['Название_text'] = family['Название']
family['Название_text'] = family['Название_text'].apply(tokenize_text)
family['Название_text']

0     [тип, исполнен, крышн, уличн, исполнен, вытяжн...
1     [крышн, вытяжн, установк, уличн, исполнен, ком...
2     [вытяжн, вентиляцион, агрегат, крышн, исполнен...
3     [вытяжн, вентиляцион, агрегат, крышн, исполнен...
4     [вентиляцион, вытяжн, установк, крышн, исполне...
5     [вентиляцион, вытяжн, установк, крышн, исполне...
6     [крышн, центробежн, вентилятор, вытяжн, состав...
7     [крышн, вентилятор, вытяжк, состав, клапа, воз...
8     [крышн, вентилятор, горизонтальн, выброс, возд...
9     [крышн, вентилятор, вертикальн, выброс, воздух...
10    [крышн, вентиляцион, установк, вытяжн, комплек...
Name: Название_text, dtype: object

In [41]:
grand['Название_text'] = grand['Название_text'].str.join(' ')
grand['Название_num'] = grand['Название_num'].str.join(' ')

family['Название_text'] = family['Название_text'].str.join(' ')
family['Название_num'] = family['Название_num'].str.join(' ')

In [42]:
grand = grand.reset_index()
family = family.reset_index()

In [44]:
grand.head()

,index,id,Хэш-код,Название,Воздухо - производительность,Напор,Название_all,Название_num,Название_text
0,0,26558,b4ceac0d-23e8-952c-e8e8-83b43d5db298,Тип исполнения крышное Вытяжной вентилятор в к...,2630,120,2630 120 Тип исполнения крышное Вытяжной венти...,2630 120 1 2 3 4,тип исполнен крышн вытяжн вентилятор комплект ...
1,1,26557,39b625d7-cd5f-68ba-64b0-a3298fbb0603,Тип исполнения подвесной Канальные установки в...,168,110,168 110 Тип исполнения подвесной Канальные уст...,168 110 1 2 3 4,тип исполнен подвесн канальн установк комплект...
2,2,26556,cbee6473-bead-d32d-97a5-437b2db61bb3,Тип исполнения подвесной Канальные установки в...,440,120,440 120 Тип исполнения подвесной Канальные уст...,440 120 1 2 3 4 5,тип исполнен подвесн канальн установк комплект...
3,3,26555,d9b50bd7-4f1c-d80e-a457-801adf2c65d5,Тип исполнения подвесной Канальные установки в...,510,160,510 160 Тип исполнения подвесной Канальные уст...,510 160 1 2 3 4 5,тип исполнен подвесн канальн установк комплект...
4,4,26554,269a68a7-1664-90a4-d96c-939518fc77d1,Тип исполнения подвесной Канальные установки в...,620,160,620 160 Тип исполнения подвесной Канальные уст...,620 160 1 2 3 4 5,тип исполнен подвесн канальн установк комплект...


### Модель

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

In [46]:
clf_text = Pipeline([     ('tfidf', TfidfVectorizer(
                                                   )
                          ),
                         ('clf', RandomForestClassifier())
                     ])
clf_num = Pipeline([     ('tfidf', TfidfVectorizer(
                                                   )
                          ),
                         ('clf', RandomForestClassifier())
                     ])
                    
  

In [47]:
clf_text.fit(grand['Название_text'], grand['index'])
clf_num.fit(grand['Название_num'], grand['index'])

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier())])

In [48]:
proba_text = clf_text.predict_proba(family['Название_text'])

all_top5_max_index_text = []
all_top5_max_value_text = []
for one_proba in proba_text:
    top5_max_index_text = np.argsort(one_proba)[-35:]
    top5_max_value_text = np.flip(one_proba[top5_max_index_text])
    
    all_top5_max_index_text.append(top5_max_index_text)
    all_top5_max_value_text.append(top5_max_value_text)
    print(top5_max_value_text)
    
all_top5_max_index_text = np.array(all_top5_max_index_text)
all_top5_max_value_text = np.array(all_top5_max_value_text)

[0.04067613 0.03970204 0.03845706 0.0376243  0.03722369 0.0369936
 0.03643682 0.03613249 0.03609236 0.0359317  0.03527973 0.03450177
 0.0340824  0.03404609 0.03378466 0.03304453 0.03241617 0.03233157
 0.031765   0.03117699 0.03117099 0.03108052 0.03074957 0.03069664
 0.02993112 0.02974872 0.02962704 0.02929631 0.01166667 0.00833333
 0.0026738  0.00208556 0.00208556 0.00181818 0.00181818]
[0.02476816 0.02456245 0.02449523 0.02431737 0.02414968 0.02409623
 0.02394146 0.02387251 0.02382681 0.02374151 0.02335494 0.02200538
 0.02194624 0.0218741  0.02143558 0.02086377 0.02074225 0.02068555
 0.02       0.01983319 0.01938774 0.01846039 0.01793746 0.01764373
 0.01739053 0.01715414 0.01690242 0.01628895 0.01432224 0.01333333
 0.01164598 0.01116667 0.01016754 0.00930658 0.00900825]
[0.04595412 0.04381886 0.04297416 0.042656   0.04230383 0.0407497
 0.04061917 0.04054872 0.03898453 0.03898323 0.03864794 0.03619414
 0.03406339 0.03350221 0.03       0.01959904 0.01843985 0.01509651
 0.01331978 0.012

In [49]:
proba_num = clf_num.predict_proba(family['Название_num'])

all_top5_max_index_num = []
all_top5_max_value_num = []
for one_proba in proba_num:
    #сортируем индексы по максимальным значения из one_proba
    top5_max_index_num = np.argsort(one_proba)[-35:]
    top5_max_value_num = np.flip(one_proba[top5_max_index_num])
    
    all_top5_max_index_num.append(top5_max_index_num)
    all_top5_max_value_num.append(top5_max_value_num)
    print(top5_max_value_num)
    
all_top5_max_index_num = np.array(all_top5_max_index_num)
all_top5_max_value_num = np.array(all_top5_max_value_num)

[0.14493573 0.13676018 0.13044447 0.12601895 0.12513298 0.12010576
 0.11346234 0.10313958 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
[0.14493573 0.13676018 0.13044447 0.12601895 0.12513298 0.12010576
 0.11346234 0.10313958 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]
[0.14493573 0.13676018 0.13044447 0.12601895 0.12513298 0.12010576
 0.11346234 0.10313958 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.

In [50]:
import xlsxwriter

In [52]:
# суммирую результаты по нум и по тексту, получается не очень
result_dict = {}
workbook = xlsxwriter.Workbook('write_data.xlsx', {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet()
i = 0
for (index, fam_row), top5_max_index_text, top5_max_value_text, top5_max_index_num, top5_max_value_num in zip(
                                                            family.iterrows(),
                                                            all_top5_max_index_text, all_top5_max_value_text,
                                                            all_top5_max_index_num, all_top5_max_value_num
                                                           ):
#     print(fam_row['ФСК_Описание'])
#     print(fam_row['id'])
    worksheet.write(i, 0, fam_row['GrandTenderHash'])
    worksheet.write(i, 1, fam_row['Название'])
    worksheet.write(i, 2, fam_row['Название_text'])
    worksheet.write(i, 3, fam_row['Название_num'])
    i += 1
    
#     print(top5_max_index_text)
#     print(top5_max_value_text)

    df_index_value_text = pd.DataFrame((top5_max_index_text, top5_max_value_text)).transpose()
    df_index_value_text.columns = ['index', 'values_text']
    
    df_index_value_num = pd.DataFrame((top5_max_index_num, top5_max_value_num)).transpose()
    df_index_value_num.columns = ['index', 'values_num']
    
#     print(df_index_value_text)
#     print('------')
#     print(df_index_value_num)
#     print('######')
    df_index_value = pd.merge(df_index_value_text, df_index_value_num, how='inner')
    # print(df_index_value.columns)
    # df_index_value = df_index_value.sort_values(by=['values_num', 'values_text'], ascending=False).iloc[:10, :]
#     print(df_index_value)
    
    df_res = pd.merge(df_index_value, grand, how='left').iloc[:10, :]

    
    if fam_row['GrandTenderHash'] in df_res['Хэш-код'].to_list():
        worksheet.write(i - 1, 4, 'YES!')
    else:
        worksheet.write(i - 1, 4, 'NO:((')

    df_res.drop(['index', 'id', "values_text", "values_num"], axis=1, inplace=True)
    # print(df_res.columns)
    cols = ['Хэш-код']  + [col for col in df_res if col != 'Хэш-код']
    df_res = df_res[cols]
    print(cols)

    j = 0
    for col in cols:
        worksheet.write(i, j, col)
        j += 1
    i += 1
    # печать содержимого df
    for string in df_res.values.tolist():
        j = 0
        for cell in string:
            worksheet.write(i, j, cell)
            j += 1  
        i += 1
    i += 1

workbook.close()

['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Название_num', 'Название_text']
['Хэш-код', 'Название', 'Воздухо - производительность', 'Напор', 'Название_all', 'Назван

In [29]:
print('perfecto')

perfecto
